## Homework 2
1. Input image from eos_map_x.png.
2. Try to detect the location of the <b>player</b> (in cyan color), and the location of the nearest <b>shrine</b> (in red color). (hint: use <i>inRange()</i>)
3. Try to detect the facing direction of the <b>player</b>. (hint: possibly use <i>morphologyEx()</i>, or <i>findContours(), minEnclosingTriangle()</i> to determine the player axis)
4. Draw a yellow line indicating the facing direction of the <b>player</b>.
5. Draw a yellow line from the <b>player</b> to the <b>shrine</b>.
4. Compute the required <b>rotating angle</b> so the player is facing to the shrine. 
(positive angle means clockwise rotation, negative angle means counterclockwise rotaion) (hint: use <i>atan2()</i>)
5. Print the rotating angle on top-left corner of the output images. (in degree, not radian)
6. Write a simple report in a separate cell.
7. Upload your Jupyter code file (*.ipynb)

In [1]:
import cv2
import numpy as np

In [2]:
# Load an image using 'imread' specifying the path to image

image = cv2.imread('eos_map_0.png')

if image is None:
    print("Error: Unable to load image.")
else:
    print("Image loaded successfully.")

Image loaded successfully.


In [3]:
# Define the range of red and blue in HSV
lower_red = np.array([0, 100, 100])
upper_red = np.array([0, 255, 255])
lower_blue = np.array([90, 100, 100])
upper_blue  = np.array([120, 255, 255])

# Convert the image to HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
red_mask = cv2.inRange(hsv, lower_red, upper_red)
blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)


# Apply the mask to the image
red_masked_image = cv2.bitwise_and(image, image, mask=red_mask)
blue_masked_image = cv2.bitwise_and(image, image, mask=blue_mask)

cv2.imshow('HSV Picture', hsv)
cv2.imshow('Original Image', image)
cv2.imshow('Red Mask', red_mask)
cv2.imshow('Red Masked Image', red_masked_image)
cv2.imshow('Blue Mask', blue_mask)
cv2.imshow('Blue Masked Image', blue_masked_image)
cv2.waitKey()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/infor/miniconda3/envs/computervision/lib/python3.9/site-packages/cv2/qt/plugins"


## Image Processing Explanation

This document provides an explanation of a code snippet used for image processing with the OpenCV library. The code is designed to isolate and analyze red and blue regions in an image by applying color filtering in the HSV color space.

### Step-by-Step Code Breakdown

1. **Define HSV Color Ranges for Red and Blue:**
   - `lower_red` and `upper_red` set the HSV range for detecting red colors. Note that the range provided may not capture all shades of red accurately.
   - `lower_blue` and `upper_blue` define the HSV range for blue colors, specifically capturing hues from 90 to 120.

   ```python
   lower_red = np.array([0, 100, 100])
   upper_red = np.array([0, 255, 255])
   lower_blue = np.array([90, 100, 100])
   upper_blue  = np.array([120, 255, 255])

2. **Convert the Image from BGR to HSV Color Space:**

   - `cv2.cvtColor` set RGB transform to HSV for image.

   ```python
   hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

3. **Create Masks for Specific Color Ranges:**
   - `cv2.inRange` checks every pixel to see if its HSV value falls within the specified range, marking it white if it does, and black otherwise. This method is efficient for segmenting images based on color.

   ```python
   red_mask = cv2.inRange(hsv, lower_red, upper_red)
   blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

4. **Apply Masks to Isolate Color Regions:**
   - `cv2.bitwise_and`are used to filter the original image, isolating areas based on color. This step is crucial for further processing, such as contour detection or region analysis

   ```python
   red_masked_image = cv2.bitwise_and(image, image, mask=red_mask)
   blue_masked_image = cv2.bitwise_and(image, image, mask=blue_mask)

5. **Display the Images:**
   - `cv2.imshow` to show our result

   ```python
   cv2.imshow('HSV Picture', hsv)
   cv2.imshow('Original Image', image)
   cv2.imshow('Red Mask', red_mask)
   cv2.imshow('Red Masked Image', red_masked_image)
   cv2.imshow('Blue Mask', blue_mask)
   cv2.imshow('Blue Masked Image', blue_masked_image)
   cv2.waitKey()
   cv2.destroyAllWindows()

In [4]:
kernel = np.ones((5,5), np.uint8)
red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)
blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_CLOSE, kernel)

### Morphological Transformations on Color Masks

Morphological transformations are key operations in image processing that process images based on shapes. They apply a structuring element to an input image and generate an output image. In this code, we specifically use the "closing" operation, which is useful for closing small holes inside the foreground objects or small black points on the object.

### Step-by-Step Code Breakdown

1. **Creating a Structuring Element:**
   - `np.ones((5,5), np.uint8)` creates a 5x5 matrix filled with ones. This matrix acts as a structuring element for the morphological operation. The size of the matrix can be adjusted depending on the specifics of the image and the desired effect.

   ```python
   kernel = np.ones((5,5), np.uint8)

2. **Applying Morphological Closing:**
    - `cv2.morphologyEx` is the function used to apply morphological transformations.
    -`cv2.MORPH_CLOSE` parameter specifies the type of operation—closing.

    ```python
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)
    blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_CLOSE, kerne)


### Purpose and Benefit

1. **Enhance Mask Quality:** Closing helps in removing small holes within detected objects in the mask, which improves the mask's quality and ensures better detection and segmentation of the desired colors.

2. **Noise Reduction:** This operation also reduces noise within the masks. It helps in eliminating small black points that can be considered as noise.

In [5]:
red_contours, _ = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
red_centers = []
red_areas = []

threshold_area = 100

for contour in red_contours:

    area = cv2.contourArea(contour)

    if area > threshold_area:

        moments = cv2.moments(contour)
        if moments["m00"] != 0:
            cx = int(moments["m10"] / moments["m00"])
            cy = int(moments["m01"] / moments["m00"])
            red_centers.append((cx, cy))  
            red_areas.append(area)       
max_red_area = max(red_areas)

for i, center in enumerate(red_centers):
    print(f"Center point {i+1}: {center}, Area: {red_areas[i]}")

print("Max Red Area:", max_red_area)

Center point 1: (83, 201), Area: 123.0
Center point 2: (117, 86), Area: 129.5
Max Red Area: 129.5


In [6]:
blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
blue_centers      = []  
blue_areas        = []

blue_threshold_area = 40

for contour in blue_contours:
    area = cv2.contourArea(contour)
    if area > blue_threshold_area:
       
        moments = cv2.moments(contour)
        if moments["m00"] != 0:
            cx = int(moments["m10"] / moments["m00"])
            cy = int(moments["m01"] / moments["m00"])
            blue_centers.append((cx, cy))
            blue_areas.append(area)

max_blue_area = max(blue_areas)

for i, center in enumerate(blue_centers):
    print(f"Center point {i+1}: {center}, Area: {blue_areas[i]}")

print("Max Blue Area:", max_blue_area)

Center point 1: (114, 115), Area: 41.0
Max Blue Area: 41.0


In [7]:
if red_centers and blue_centers:
    
    min_dist = float('inf')
    closest_red  = None
    closest_blue = None
    
    for red_center in red_centers:
        for blue_center in blue_centers:
            dist = np.linalg.norm(np.array(blue_center) - np.array(red_center)) # 计算向量长度
        
            if dist < min_dist:
                min_dist = dist
                closest_red  = red_center 
                closest_blue = blue_center 

    closest_red  = tuple(closest_red)
    closest_blue = tuple(closest_blue)

    img_with_line = cv2.line(image.copy(), closest_red, closest_blue, (0, 255, 255),2)

    cv2.imshow('Image with Line', img_with_line)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [8]:
epsilon = 0.1 * cv2.arcLength(blue_contours[0], True)
approx = cv2.approxPolyDP(blue_contours[0], epsilon, True)

In [9]:
hull = cv2.convexHull(approx)
hull_points = [tuple(point[0]) for point in hull]

In [10]:
side_length_ab =  ((hull_points[0][0] - hull_points[1][0])** 2 + (hull_points[0][1] - hull_points[1][1])** 2 ) ** 0.5
side_length_ac =  ((hull_points[0][0] - hull_points[2][0])** 2 + (hull_points[0][1] - hull_points[2][1])** 2 ) ** 0.5
side_length_bc =  ((hull_points[1][0] - hull_points[2][0])** 2 + (hull_points[1][1] - hull_points[2][1])** 2 ) ** 0.5

In [11]:
max_side_length = max(side_length_ab + side_length_ac , side_length_bc + side_length_ac, side_length_ab + side_length_bc)

if max_side_length == side_length_ab + side_length_ac: #a
    intersection = hull_points[0]
    
if max_side_length == side_length_ab + side_length_bc: #b
    intersection = hull_points[1]
    
if max_side_length == side_length_ac + side_length_bc: #c
    intersection = hull_points[2]


In [12]:
centroid_to_intersection_vector = np.array([intersection[0] - cx, intersection[1] - cy])

extended_point = (int(cx + 3 * centroid_to_intersection_vector[0]), int(cy + 3 * centroid_to_intersection_vector[1]))


img_with_yellow_lines = cv2.line(image.copy(), (cx, cy), intersection, (0, 255, 255), 2)


img_with_line = cv2.line(img_with_line, intersection, extended_point, (0, 255, 255), 2)
img_with_line = cv2.line(img_with_line, (cx, cy), intersection, (0, 255, 255), 2)

cv2.circle(img_with_yellow_lines, intersection, 2, (0, 255, 0), -1)
cv2.circle(img_with_yellow_lines, blue_center, 2, (255, 0, 0), -1)

for red_center in red_centers:
    cv2.circle(img_with_yellow_lines, red_center, 2, (0, 0, 255), -1)

print("Intersection point coordinates:", intersection)
print("Blue center coordinates:", blue_center)
print("red center coordinates:", red_center)

vec_blue = np.array(intersection) - np.array(blue_center)
vec_red  = np.array(red_center)   - np.array(blue_center)

angle_between = np.degrees(np.arctan2(vec_red[1], vec_red[0]) - np.arctan2(vec_blue[1], vec_blue[0]))

if angle_between > 180:
    angle_between -= 360
elif angle_between < -180:
    angle_between += 360


print("Angle:", angle_between)

cv2.putText(img_with_line, f'{angle_between:.0f}', (5, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)


Intersection point coordinates: (119, 111)
Blue center coordinates: (114, 115)
red center coordinates: (117, 86)
夾角: -45.43405063213941


array([[[ 69, 147, 175],
        [ 74, 149, 178],
        [ 82, 148, 184],
        ...,
        [ 77, 106, 134],
        [ 77, 106, 134],
        [ 70, 101, 128]],

       [[ 72, 148, 177],
        [ 74, 149, 178],
        [ 82, 149, 187],
        ...,
        [ 77, 109, 136],
        [ 77, 109, 136],
        [ 77, 119, 149]],

       [[ 82, 151, 184],
        [ 82, 151, 184],
        [ 82, 150, 189],
        ...,
        [ 78, 109, 138],
        [ 82, 124, 155],
        [ 82, 130, 161]],

       ...,

       [[ 99, 158, 206],
        [ 99, 158, 206],
        [ 87, 159, 204],
        ...,
        [ 66, 127, 161],
        [ 66, 116, 152],
        [ 73, 130, 164]],

       [[ 90, 157, 206],
        [ 93, 161, 206],
        [ 82, 155, 194],
        ...,
        [ 71, 132, 172],
        [ 67, 117, 160],
        [ 71, 126, 169]],

       [[ 91, 154, 192],
        [ 87, 150, 192],
        [ 77, 144, 181],
        ...,
        [ 63, 115, 155],
        [ 63, 116, 155],
        [ 67, 119, 159]]

In [13]:
img_with_line = cv2.resize(img_with_line, None, fx=3, fy=3)
cv2.imshow('Image with Line', img_with_line)
cv2.waitKey(0)
cv2.destroyAllWindows()
